# Problem formulation
- The *Drives A/S* is a company spceilized in producing different types of drives for variable speed motor control or power conversion applications. They have a wide range of Variable Frequency Drives(VFD) to offer.
- The company wants to optimize its profits and at the same time minimize their operating costs.

## Goals/ Objectives

- Maximize the profit
- Minimize costs
- Minimize working hours
- Stay inside the budget

In [514]:
# IMPORTS
from pyomo.opt import SolverFactory
import sys
from pyomo.environ import *
import pyomo.environ as pyo
import pandas as pd

# Data

In [515]:
#mydata = pd.read_csv("C:\\BSc in EIB\\5. Semester\\Repos\Repo - Optimization and Analytics\\dt1.csv",)
mydata = pd.read_csv('dt1.csv')
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_seq_items', None)
mydata

C:\Users\Dimi\AppData\Local\Temp\ipykernel_13032\3494933311.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,VFD-Type Nr.,Sell-Price,Material cost in EUR,Net-Profit,Labour-Time per Unit(in Minutes),Total demand (in Units)
0,1785,60,24,36,19,180
1,1328,80,23,57,17,140
2,1927,128,18,110,10,136
3,1719,137,21,116,11,147
4,1849,108,23,85,20,150
5,1149,92,15,77,15,168
6,1484,119,17,102,19,133
7,1565,79,22,57,19,169
8,1435,104,24,80,13,130
9,1868,145,19,126,20,145


In [516]:
mydata.rename(columns=lambda c: c.lower().replace('-','_'), inplace=True)

In [517]:
# Create python lists for each category
type_number_array = []
for i in mydata.iloc[:,0]:
  type_number_array.append(i)
  
sell_price_array = []
for i in mydata.iloc[:,1]:
  sell_price_array.append(float(i))

material_cost_array = []
for i in mydata.iloc[:,2]:
  material_cost_array.append(float(i))

net_profit_array = []
for i in mydata.iloc[:,3]:
  net_profit_array.append(float(i))

labour_hours_array = []
for i in mydata.iloc[:,4]:
  labour_hours_array.append(float(i))

total_demand_array = []
for i in mydata.iloc[:,5]:
  total_demand_array.append(float(i))

In [518]:
L = range(len(total_demand_array))
print (L)
L

range(0, 22)


range(0, 22)

In [519]:
# Create model object
model = ConcreteModel('Profit Maximisation')

# Get length of model
#L = range(len(model.x))

# # Define unit type
# for i in L:
#   model.i = Set(initialize=type_number_array[i], doc = 'Type of Unit')
# # Define number of units
# model.j = Set(initialize='Number', doc = 'Number of units')

# # Define x(i,j)
# model.x = Var(model.i, model.j, doc = 'Number of Units produced', domain = Binary)

# Define demand
#model.dem = Param(model.j, initialize = total_demand_array[i], doc = 'Demand')

In [520]:

sell_price = {
  'A': 35,
  'B': 98,
  'C': 57,
  'D': 27,
  'E': 30,
  'F': 74,
  'G': 32,
  'H': 64,
  'I': 96,
  'J': 93,
  'K': 53,
  'L': 51,
  'M': 37,
  'N': 90,
  'O': 22,
  'P': 65,
  'Q': 69,
  'R': 30,
  'S': 78,
  'T': 52,
  'U': 86,
  'V': 60,
}

## Variables
- The company has a total range of 21 different types of VDF.
- They are all customizable and can be each produced in a timewindow of 10-25mins.

In [521]:
# Each Variable -> Number of each VFD
model.x = Var([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21], domain=Binary, doc = 'Number of Units produced')

## Constraints
- The company has a production plan for the year. With an estimated maximum demand for the units.
- The maximum amount of labour hours is 360.000min = 6000 hours per year. Which is 30.000min = 500h per month.
- The budget of the company is roughly 10.000EUR per month.
- Each unit has to be produced at least once.

In [522]:
# Labour hours
# #Type1 * LH_Type1 + #Type2 * LH_Type2 + ... <= 30.000 minutes
for i in L:
   model.cons1 = Constraint(expr = model.x[i+1]*labour_hours_array[i] <= 30000)

# Material costs
# #Type1*Mc_Type1 + #Type2 * Mc_Type2 + ... <= 10.000 EUR
   model.cons2 = Constraint(expr = model.x[i+1]*material_cost_array[i] <= 10000)

# Total demand
# Each VFD has to stay under its max demand
   #model.cons3 = Constraint(expr = model.x[i+1] * total_demand_array[i])

# Each Type has to be produced at least once
  #Type >= 0

    'pyomo.core.base.constraint.ScalarConstraint'>) on block Profit
    Maximisation with a new Component (type=<class
    'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.ScalarConstraint'>) on block Profit
    Maximisation with a new Component (type=<class
    'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.ScalarConstraint'>) on block Profit
    Maximisation with a new Component (type=<class
    'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.ScalarConstraint'>) on block 

KeyError: "Index '22' is not valid for indexed component 'x'"

In [ ]:
# def Labour_hours_cons(model, i):
#   return sum(model.x[i+1]*labour_hours_array[i])<= 60000
# model.cons1 = Constraint(model.x, rule = Labour_hours_cons, doc ='Maximum number of Labour hours')

# def Budget_cons(model,i):
#   return sum(model.x[i+1]*material_cost_array[i])<= 100000
# model.cons2 = Constraint(model.x, rule = Budget_cons, doc ='Maximum budget')

# def Total_demand_cons(model, i):
#   return sum(model.x[i+1] <= total_demand_array[i])
# model.cons3 = Constraint(model.x, rule = Total_demand_cons, doc =' Total demand')

# Objective and Solver

In [ ]:
# Objective function
# P = #Type1 * NetProfitType1 + #Type2 * NetProfitType2 + ...

#model.objective = Objective(expr = sum(model.x[i+1]*net_profit_array[i] for i in L),sense=maximize,doc='Max Profit')

#model.OBJ = Objective(rule=sum(model.x[i+1]*net_profit_array[i] for i in L),sense=maximize,doc='Max Profit')

def objective_rule(model):
  return sum(model.x[i+1]*net_profit_array[i] for i in L)
model.objective = Objective(rule=objective_rule, sense=maximize, doc='Profit Maximisation')

# Solver object
solvername='glpk'
solverpath_folder='C:\\w64'
solverpath_exe='C:\\w64\\glpsol'

sys.path.append(solverpath_folder)
solver=SolverFactory(solvername, executable=solverpath_exe)

# solve
results = solver.solve(model)
results.write()
if results.solver.status:
    model.pprint()

#display solution
print('\nProfit = ', model.objective())

print('\nDecision Variables')
for i in L:
  print('x = ', model.x[i+1])


print('\nConstraints')
print('Labour hours  = ', model.cons1())
print('Material costs = ', model.cons2())
print('Total demand = ', model.cons3())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1571.0
  Upper bound: 1571.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 22
  Number of nonzeros: 4
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.04412269592285156
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------

In [ ]:
model.objective()

1571.0